In [1]:
from urllib.request import urlretrieve
import requests
import pandas as pd

In [2]:
experiment_summary = pd.ExcelFile('/mnt/c/Users/victo/Documents/DecryptM/experiment_summary.xlsx')
experiment_summary_phospho = pd.read_excel(experiment_summary, 'PTMs')


In [4]:
rawfiles = experiment_summary_phospho['Raw file'][
    (experiment_summary_phospho['Drug'].isin(['Afatinib', 'Dasatinib', 'Gefitinib'])) & 
    (experiment_summary_phospho['Treatment time'] == '30min') & 
    (experiment_summary_phospho['Cell line'] == 'A431') &
    (experiment_summary_phospho["Search"].str.contains('EGFR'))
    ].tolist()
len(rawfiles)

72

In [5]:
rawfiles = ['04160_GD8_P046464_S00_I00_U02_R1_TMT11.raw', '04160_GD9_P046464_S00_I00_U03_R1_TMT11.raw']

In [46]:
import os
os.mkdir('/mnt/c/Users/victo/Documents/DecryptM/all_phospho')

In [6]:
for rawfile in rawfiles:
    print(f'getting {rawfile}')
    url = f"https://ftp.pride.ebi.ac.uk/pride/data/archive/2023/03/PXD037285/{rawfile}"
    filename = f"/mnt/c/Users/victo/Documents/DecryptM/all_phospho/{rawfile}"

    try:
        resp = requests.get(url).content
        with open(filename, "wb") as f:
            f.write(resp)
        print(f"{rawfile} is saved")
    except Exception as e:
        print(e)


getting 04160_GD8_P046464_S00_I00_U02_R1_TMT11.raw
('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
getting 04160_GD9_P046464_S00_I00_U03_R1_TMT11.raw
04160_GD9_P046464_S00_I00_U03_R1_TMT11.raw is saved


## Proteome


In [ ]:
import omnipath as op
import numpy as np
import pandas as pd
import decoupler as dc

/home/victo/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
collectri = dc.get_collectri()
omnipath_og = op.interactions.AllInteractions.get('omnipath',genesymbols=True)
omnipath = omnipath_og

In [ ]:
omnipath.rename(columns={'source':'source_uniprot', 'target':'target_uniprot'}, inplace=True)
omnipath.rename(columns={'source_genesymbol':'source', 'target_genesymbol':'target'}, inplace=True)



In [ ]:
# get only directed interactions
omnipath = omnipath[(omnipath['consensus_direction'] == True) & (((omnipath['consensus_stimulation'] == True) & (omnipath['consensus_inhibition'] == False)) | ((omnipath['consensus_stimulation'] == False) & (omnipath['consensus_inhibition'] == True))) & (omnipath['curation_effort'] >= 2)]

# remove interactions downstream of TFs
collectri_tfs = collectri.source.unique().tolist()
omnipath = omnipath[~omnipath['source'].isin(collectri_tfs)]


# subset the collectri GRN to enrich only TFs present in the network
collectri_tfs_in_omnipath = set(omnipath['target'][omnipath['target'].isin(collectri_tfs)])
collectri_tfs_in_omnipath_df = collectri[collectri['source'].isin(collectri_tfs_in_omnipath)]
collectri_tfs_in_omnipath_df.to_csv('collectri.tsv', sep='\t', index=False)
collectri_tfs_in_omnipath_df

omnipath['data'] = np.where(omnipath['consensus_stimulation'] == True, 1, -1)

# write the resulting omnipath network in networkx format
net = omnipath[['source', 'target', 'data']]
net.rename(columns={'source':'# source'}, inplace=True)
net.to_csv('network_collectri.sif', sep = '\t', index = False)

/tmp/ipykernel_3548/1967301379.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  net.rename(columns={'source':'# source'}, inplace=True)


In [ ]:
omnipath_og

,source_uniprot,target_uniprot,source,target,is_directed,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,curation_effort,references,sources,type,n_sources,n_primary_sources,n_references,references_stripped
0,P0DP23,P48995,CALM1,TRPC1,True,False,True,True,False,True,3,TRIP:11290752;TRIP:11983166;TRIP:12601176,TRIP,post_translational,1,1,3,11290752;11983166;12601176
1,P0DP25,P48995,CALM3,TRPC1,True,False,True,True,False,True,3,TRIP:11290752;TRIP:11983166;TRIP:12601176,TRIP,post_translational,1,1,3,11290752;11983166;12601176
2,P0DP24,P48995,CALM2,TRPC1,True,False,True,True,False,True,3,TRIP:11290752;TRIP:11983166;TRIP:12601176,TRIP,post_translational,1,1,3,11290752;11983166;12601176
3,Q03135,P48995,CAV1,TRPC1,True,True,False,True,True,False,13,DIP:19897728;HPRD:12732636;IntAct:19897728;Lit...,DIP;HPRD;IntAct;Lit-BM-17;TRIP,post_translational,5,5,8,10980191;12732636;14551243;16822931;18430726;1...
4,P14416,P48995,DRD2,TRPC1,True,True,False,True,True,False,1,TRIP:18261457,TRIP,post_translational,1,1,1,18261457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81524,P12931,Q9UJM3,SRC,ERRFI1,True,False,False,False,False,False,2,ProtMapper:26280531;iPTMnet:26280531,ProtMapper;RLIMS-P_ProtMapper;iPTMnet,post_translational,3,2,1,26280531
81525,P43405,Q9ULZ3,SYK,PYCARD,True,False,False,False,False,False,2,ProtMapper:25605870;iPTMnet:25605870,ProtMapper;RLIMS-P_ProtMapper;iPTMnet,post_translational,3,2,1,25605870
81526,Q14289,Q9ULZ3,PTK2B,PYCARD,True,False,False,False,False,False,2,ProtMapper:27796369;iPTMnet:27796369,ProtMapper;RLIMS-P_ProtMapper;iPTMnet,post_translational,3,2,1,27796369
81527,P54646,Q9Y2I7,PRKAA2,PIKFYVE,True,False,False,False,False,False,1,iPTMnet:24070423,iPTMnet,post_translational,1,1,1,24070423


In [ ]:
uniprot_tfs = set(omnipath_og['target_uniprot'][omnipath_og['target'].isin(collectri_tfs_in_omnipath)])

In [ ]:
# open a file and write all the fasta sequences in it
with open('uniprot_tfs.fasta', 'w') as f:
    for uniprot_id in uniprot_tfs:
        print(uniprot_id)
        url = f'https://rest.uniprot.org/uniprotkb/{uniprot_id}?format=fasta'
        fasta = urlopen(url).read().decode('utf-8')
        f.write(fasta)

In [ ]:
from urllib.request import urlopen

In [ ]:
# open this url and print the output
x = urlopen('https://rest.uniprot.org/uniprotkb/O15266?format=fasta')

b'>sp|O15266|SHOX_HUMAN Short stature homeobox protein OS=Homo sapiens OX=9606 GN=SHOX PE=1 SV=1\nMEELTAFVSKSFDQKSKDGNGGGGGGGGKKDSITYREVLESGLARSRELGTSDSSLQDIT\nEGGGHCPVHLFKDHVDNDKEKLKEFGTARVAEGIYECKEKREDVKSEDEDGQTKLKQRRS\nRTNFTLEQLNELERLFDETHYPDAFMREELSQRLGLSEARVQVWFQNRRAKCRKQENQMH\nKGVILGTANHLDACRVAPYVNMGALRMPFQQVQAQLQLEGVAHAHPHLHPHLAAHAPYLM\nFPPPPFGLPIASLAESASAAAVVAAAAKSNSKNSSIADLRLKARKHAEALGL\n'
